In [37]:
# Standard library imports
import sys
import sqlite3

# Related third party imports
from tqdm import tqdm
import pandas as pd

# Local application/library specific imports.
# paradoxcell
sys.path.append('/10Gdata/sjt')
import paradoxcell as pc

In [38]:
connect = sqlite3.connect('/10Gdata/sjt/data/database/eggnog/eggnog-mapper-data/eggnog.db')
cursor = connect.cursor()
cursor.execute("SELECT * FROM event WHERE level=33090;")
tuple_eventtable_from_eggnogdb_where_level33090 = cursor.fetchall()

list_gene_og = [",".join([k,og[2]]) for og in tuple_eventtable_from_eggnogdb_where_level33090 for i in og[3:] for k in i.split(',')]
list_gene_og = list(set(list_gene_og))
list_gene_og = sorted(list_gene_og)

In [44]:
dic_gene_og = dict()
for i in list_gene_og:
    key = i.split(',')[0]
    value = i.split(',')[1]
    dic_gene_og[key] = value

In [42]:
genes = [i.split(',')[0] for i in list_gene_og]

In [ ]:
df_stringid_uniprotid = pd.DataFrame()

for i in range(0,len(genes)+1,50000):
    if i+5000 < len(genes):
        df = pc.get_df_uniprot_mapping('STRING_ID','ID'," ".join(genes[i:i+50000]))
        df_stringid_uniprotid = pd.concat([df_stringid_uniprotid,df])
    else:
        df = pc.get_df_uniprot_mapping('STRING_ID','ID'," ".join(genes[i:]))
        df_stringid_uniprotid = pd.concat([df_stringid_uniprotid,df])

df_stringid_uniprotid

In [ ]:
df_uniprotid_stringid_ensemblgenomeid_og_for_eggnog33090

In [ ]:
df_uniprotid_ensemblgenomeid = pd.DataFrame()

for i in range(0,len(df_stringid_uniprotid['To'])+1,50000):
    if i+5000 < len(df_stringid_uniprotid['To']):
        df = pc.get_df_uniprot_mapping('ID','ENSEMBLGENOME_ID'," ".join(df_stringid_uniprotid['To'][i:i+50000]))
        df_uniprotid_ensemblgenomeid = pd.concat([df_uniprotid_ensemblgenomeid,df])
    else:
        df = pc.get_df_uniprot_mapping('ID','ENSEMBLGENOME_ID'," ".join(df_stringid_uniprotid['To'][i:]))
        df_uniprotid_ensemblgenomeid = pd.concat([df_uniprotid_ensemblgenomeid,df])

df_uniprotid_ensemblgenomeid

In [ ]:
df_stringid_uniprotid.rename(columns={'From':'stringid','To':'uniprotid'},inplace=True)
df_uniprotid_ensemblgenomeid.rename(columns={'From':'uniprotid','To':'ensemblgenomeid'},inplace=True)

In [ ]:
df_uniprotid_stringid_ensemblgenomeid_og_for_eggnog33090 = pd.merge(df_stringid_uniprotid,df_uniprotid_ensemblgenomeid,on='uniprotid')
df_uniprotid_stringid_ensemblgenomeid_og_for_eggnog33090['og'] = df_uniprotid_stringid_ensemblgenomeid_og_for_eggnog33090['stringid'].apply(lambda x : dic_gene_og[x])
df_uniprotid_stringid_ensemblgenomeid_og_for_eggnog33090